In [1]:
import numpy as np

sequence1="ACCGACTAGACAGGT"
sequence2="TTACCGGACTAGACCAGCG"
sequence3="ACAATCG"
sequence4="CTCATGC"
bases=["-","A","C","G","T"]

score_Table=[[0,-1,-1,-1,-1],[-1,2,-1,-1,-1],[-1,-1,2,-1,-1],[-1,-1,-1,2,-1],[-1,-1,-1,-1,2]]

score_Table_Dict={"-":{"-":0, "A":0,"C":0,"G":0,"T":0}
             , "A":{"-":0, "A":0,"C":0,"G":0,"T":0}
             ,"C":{"-":0, "A":0,"C":0,"G":0,"T":0}
             ,"G":{"-":0, "A":0,"C":0,"G":0,"T":0}
             ,"T":{"-":0, "A":0,"C":0,"G":0,"T":0}
            }
#print(score_Table_Dict)

def fill_score_table(array, score_Table):
    array=np.array(array)
    keys=list(score_Table.keys())
    for i in range(array.shape[0]):
        for j in range(array.shape[1]):
            score_Table[keys[i]][keys[j]]=array[i][j]
    
    return score_Table
score_Table_Dict= fill_score_table(score_Table, score_Table_Dict)

def local_alignment(sequence1, sequence2, scoreTable):
    alignment_table=np.zeros((len(sequence2)+1,len(sequence1)+1))
    alignment_table[0][0]=0
    
    
    # init first line
    for i in range(1,len(sequence1)+1,1):
        alignment_table[0][i]= alignment_table[0][i-1]+scoreTable['-'][sequence1[i-1]]
        alignment_table[0][i]= np.maximum(alignment_table[0][i],0)
        
    # init first column
    for i in range(1, len(sequence2)+1,1):
        alignment_table[i][0]= alignment_table[i-1][0]+ scoreTable[sequence2[i-1]]['-']
        alignment_table[i][0]= np.maximum(alignment_table[i][0],0)
    
    for i in range(1,len(sequence2)+1,1):
        for j in range(1,len(sequence1)+1,1):
            upper= alignment_table[i-1][j]+ scoreTable[sequence2[i-1]]["-"]              # i-1 j
            left=  alignment_table[i][j-1]+ scoreTable["-"][sequence1[j-1]]               #i j-1
            diagonale= alignment_table[i-1][j-1]+ scoreTable[sequence2[i-1]][sequence1[j-1]]            #i-1 j-1
            alignment_table[i][j]=max([upper,left,diagonale,0])
            if(j==2 and i==1):
                print(scoreTable)
                print(upper)
                print(diagonale)
                print(left)
                print(alignment_table[i][j-1])
                print(sequence2[i-1])
                print(sequence1[j-1])
    print(scoreTable)
    return alignment_table
                
                
            
alignmentMatrix=local_alignment(sequence3, sequence4, score_Table_Dict)    
print(alignmentMatrix)

{'-': {'-': 0, 'A': -1, 'C': -1, 'G': -1, 'T': -1}, 'A': {'-': -1, 'A': 2, 'C': -1, 'G': -1, 'T': -1}, 'C': {'-': -1, 'A': -1, 'C': 2, 'G': -1, 'T': -1}, 'G': {'-': -1, 'A': -1, 'C': -1, 'G': 2, 'T': -1}, 'T': {'-': -1, 'A': -1, 'C': -1, 'G': -1, 'T': 2}}
-1.0
2.0
-1.0
0.0
C
C
{'-': {'-': 0, 'A': -1, 'C': -1, 'G': -1, 'T': -1}, 'A': {'-': -1, 'A': 2, 'C': -1, 'G': -1, 'T': -1}, 'C': {'-': -1, 'A': -1, 'C': 2, 'G': -1, 'T': -1}, 'G': {'-': -1, 'A': -1, 'C': -1, 'G': 2, 'T': -1}, 'T': {'-': -1, 'A': -1, 'C': -1, 'G': -1, 'T': 2}}
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 1. 0. 0. 2. 1.]
 [0. 0. 1. 1. 0. 2. 1. 1.]
 [0. 0. 2. 1. 0. 1. 4. 3.]
 [0. 2. 1. 4. 3. 2. 3. 3.]
 [0. 1. 1. 3. 3. 5. 4. 3.]
 [0. 0. 0. 2. 2. 4. 4. 6.]
 [0. 0. 2. 1. 1. 3. 6. 5.]]


In [26]:
def get_max_score_indices(alignment_matrix):
    indices=[]
    maxScore=float('-inf')
    for i in range(alignment_matrix.shape[0]):
        for j in range(alignment_matrix.shape[1]):
            if(alignment_matrix[i][j]>maxScore):
                maxScore=alignment_matrix[i][j]
                I=i
                J=j
    for i in range(alignment_matrix.shape[0]):
        for j in range(alignment_matrix.shape[1]):
            if(alignment_matrix[i][j]==maxScore):
                indices.append([i,j])
    return indices
            
indeledSequence1="-"+sequence3
indeledSequence2="-"+sequence4

def optimal_path(i,j, alignment_matrix,path,depth):
    print(i,j,depth)
    path.append([i,j])
    if(i==0 and j==0):
        return alignment_matrix[i][j]
    if(i==0 and j!=0):
        if(j==1):
            return {'score':optimal_path(i,j-1, alignment_matrix,path[:],depth+1),'path':path}
        else:
            return {'score':optimal_path(i,j-1, alignment_matrix,path[:],depth+1)['score']+alignment_matrix[i][j],'path':path[:]}
    if(j==0 and i!=0):
        if(i==1):
            return {'score':optimal_path(i-1,j, alignment_matrix,path[:],depth+1),'path':path}
        else:
            return {'score':optimal_path(i-1,j,alignment_matrix,path[:],depth+1)['score']+alignment_matrix[i][j],'path':path[:]}
    if(i>0 and j>0):
        up=optimal_path(i-1,j,alignment_matrix,path[:],depth+1)
        left=optimal_path(i,j-1,alignment_matrix,path[:],depth+1)
        diag=optimal_path(i-1,j-1,alignment_matrix,path[:],depth+1)
        print(i,j)
        print('up',up,'left',left,'diag',diag, 'depth=',depth)
        #index=np.argmax([up[0],left[0],diag[0]])
        if(depth==0):
            return path
        else:
            
            return {'score':max(up['score'],left['score'] ,diag['score'])+alignment_matrix[i][j],'path':path[:]}

        
def optimal_path2(i,j, alignment_matrix,path,depth, indeledSequence1, indeledSequence2,scoreTable):
    residue2=indeledSequence2[i]
    residue1=indeledSequence1[j]
    print(i,j,depth)
    path.append([i,j])
    
    if(i==0 and j==0):
        return
    if(i==0):
        if(np.max([(alignment_matrix[i][j-1]+ scoreTable["-"][indeledSequence1[j-1+1]]),0]==alignment_matrix[i][j])):
            optimal_path2(i,j-1, alignment_matrix,path,depth+1,indeledSequence1,indeledSequence2,scoreTable)
        else:
            return
    if(j==0):
        if(np.max([(alignment_matrix[i-1][j]+ scoreTable[indeledSequence2[i-1+1]]["-"]),0]==alignment_matrix[i][j])):
            optimal_path2(i-1,j,alignment_matrix,path,depth+1,indeledSequence1,indeledSequence2,scoreTable)
        else:
            return
    if(i>0 and j>0):
        print(np.max([(alignment_matrix[i][j-1]+ scoreTable["-"][indeledSequence1[j-1+1]]),0]),alignment_matrix[i][j])
        print(np.max([(alignment_matrix[i-1][j]+ scoreTable[indeledSequence2[i-1+1]]["-"]),0]),alignment_matrix[i][j])
        print(np.max([(alignment_matrix[i-1][j-1]+ scoreTable[indeledSequence2[i-1+1]][indeledSequence1[j-1+1]] ),0]),alignment_matrix[i][j])
        
        if(np.max([(alignment_matrix[i][j-1]+ scoreTable[indeledSequence1[j-1+1]]["-"]),0])==alignment_matrix[i][j]):
            optimal_path2(i,j-1, alignment_matrix,path,depth+1,indeledSequence1,indeledSequence2,scoreTable)
            if(depth==0):
                return path
            else:
                return
        if(np.max([(alignment_matrix[i-1][j]+ scoreTable["-"][indeledSequence2[i-1+1]]),0])==alignment_matrix[i][j]):
            optimal_path2(i-1,j,alignment_matrix,path,depth+1,indeledSequence1,indeledSequence2,scoreTable)
            if(depth==0):
                return path
            else:
                return
        if(np.max([(alignment_matrix[i-1][j-1]+ scoreTable[indeledSequence2[i-1+1]][indeledSequence1[j-1+1]] ),0])==alignment_matrix[i][j]):
            optimal_path2(i-1,j-1,alignment_matrix,path,depth+1,indeledSequence1,indeledSequence2,scoreTable)
            if(depth==0):
                return path
            else:
                return

        

        
def optimal_path3(i,j, alignment_matrix,path,depth, indeledSequence1, indeledSequence2,scoreTable):
    residue2=indeledSequence2[i]
    residue1=indeledSequence1[j]
    print(i,j,depth)
    path.append([i,j])
    
    if(i==0 and j==0):
        return
    if(i==0):
        optimal_path3(i,j-1, alignment_matrix,path,depth+1,indeledSequence1,indeledSequence2,scoreTable)
        return
    if(j==0):
        optimal_path3(i-1,j,alignment_matrix,path,depth+1,indeledSequence1,indeledSequence2,scoreTable)
        return
    if(i>0 and j>0):
        up=alignment_matrix[i-1][j]
        left=alignment_matrix[i][j-1]
        diag=alignment_matrix[i-1][j-1]
        maxIndx=np.argmax([up,left,diag])
        if(up==0 and left==0 and diag==0):
            return
        print('argmax=',maxIndx)
        print([up,left,diag])
        if(maxIndx==0):
            optimal_path3(i-1,j, alignment_matrix,path,depth+1,indeledSequence1,indeledSequence2,scoreTable)
        if(maxIndx==1):
            optimal_path3(i,j-1, alignment_matrix,path,depth+1,indeledSequence1,indeledSequence2,scoreTable)
        if(maxIndx==2):
            optimal_path3(i-1,j-1, alignment_matrix,path,depth+1,indeledSequence1,indeledSequence2,scoreTable)
        if(depth==0):
            return path
        else:
            return

max_indices=get_max_score_indices(alignmentMatrix)
optimale_scores=[]
for maxScore in max_indices:
    optimale_scores.append(optimal_path2(maxScore[0],maxScore[1],alignmentMatrix,[],0,indeledSequence1,indeledSequence2,score_Table_Dict))

print(optimale_scores)

IndexError: string index out of range

In [3]:
import blosum as bl
matrix = bl.BLOSUM(62)

In [27]:
def fill_Score_From_Blosum(blosum):
    residueSet=[]
    scoreTable_Dict={}
    
    for key in blosum.keys():
        residue1=key[0].replace('*','-')
        residue2=key[1].replace('*','-')
        if(residue1 not in residueSet):
            residueSet.append(residue1)
        if(residue2 not in residueSet):
            residueSet.append(residue2)
            
    for residue in residueSet:
        scoreTable_Dict[residue]={}
        
    
    for key in blosum.keys():
        residue1=key[0].replace('*','-')
        residue2=key[1].replace('*','-')
        scoreTable_Dict[residue1][residue2]=blosum[key]
    return scoreTable_Dict

score_table_blosum=fill_Score_From_Blosum(matrix)

In [28]:
alignmentMatrix=local_alignment(sequence1, sequence2, score_table_blosum) 
print(alignmentMatrix)

{'A': {'A': 4.0, 'R': -1.0, 'N': -2.0, 'D': -2.0, 'C': 0.0, 'Q': -1.0, 'E': -1.0, 'G': 0.0, 'H': -2.0, 'I': -1.0, 'L': -1.0, 'K': -1.0, 'M': -1.0, 'F': -2.0, 'P': -1.0, 'S': 1.0, 'T': 0.0, 'W': -3.0, 'Y': -2.0, 'V': 0.0, 'B': -2.0, 'J': -1.0, 'Z': -1.0, 'X': -1.0, '-': -4.0}, 'R': {'A': -1.0, 'R': 5.0, 'N': 0.0, 'D': -2.0, 'C': -3.0, 'Q': 1.0, 'E': 0.0, 'G': -2.0, 'H': 0.0, 'I': -3.0, 'L': -2.0, 'K': 2.0, 'M': -1.0, 'F': -3.0, 'P': -2.0, 'S': -1.0, 'T': -1.0, 'W': -3.0, 'Y': -2.0, 'V': -3.0, 'B': -1.0, 'J': -2.0, 'Z': 0.0, 'X': -1.0, '-': -4.0}, 'N': {'A': -2.0, 'R': 0.0, 'N': 6.0, 'D': 1.0, 'C': -3.0, 'Q': 0.0, 'E': 0.0, 'G': 0.0, 'H': 1.0, 'I': -3.0, 'L': -3.0, 'K': 0.0, 'M': -2.0, 'F': -3.0, 'P': -2.0, 'S': 1.0, 'T': 0.0, 'W': -4.0, 'Y': -2.0, 'V': -3.0, 'B': 4.0, 'J': -3.0, 'Z': 0.0, 'X': -1.0, '-': -4.0}, 'D': {'A': -2.0, 'R': -2.0, 'N': 1.0, 'D': 6.0, 'C': -3.0, 'Q': 0.0, 'E': 2.0, 'G': -1.0, 'H': -1.0, 'I': -3.0, 'L': -4.0, 'K': -1.0, 'M': -3.0, 'F': -3.0, 'P': -1.0, 'S': 0.0, '

In [29]:
indeledSequence1="-"+sequence1
indeledSequence2="-"+sequence2
max_indices=get_max_score_indices(alignmentMatrix)
print(max_indices)
optimale_scores=[]
for maxScore in max_indices:
        optimale_scores.append(optimal_path3(maxScore[0],maxScore[1],alignmentMatrix,[],0,indeledSequence1,indeledSequence2,score_table_blosum))

print(optimale_scores)

[[19, 14]]
19 14 0
argmax= 0
[68.0, 63.0, 67.0]
18 14 1
argmax= 0
[71.0, 67.0, 71.0]
17 14 2
argmax= 1
[61.0, 71.0, 65.0]
17 13 3
argmax= 0
[65.0, 61.0, 65.0]
16 13 4
argmax= 1
[61.0, 65.0, 65.0]
16 12 5
argmax= 0
[65.0, 57.0, 61.0]
15 12 6
argmax= 2
[61.0, 61.0, 65.0]
14 11 7
argmax= 2
[52.0, 52.0, 56.0]
13 10 8
argmax= 2
[48.0, 48.0, 52.0]
12 9 9
argmax= 2
[42.0, 42.0, 46.0]
11 8 10
argmax= 2
[38.0, 38.0, 42.0]
10 7 11
argmax= 2
[33.0, 33.0, 37.0]
9 6 12
argmax= 0
[28.0, 24.0, 28.0]
8 6 13
argmax= 1
[24.0, 28.0, 28.0]
8 5 14
argmax= 0
[28.0, 20.0, 24.0]
7 5 15
argmax= 2
[24.0, 24.0, 28.0]
6 4 16
argmax= 2
[18.0, 18.0, 22.0]
5 3 17
argmax= 2
[9.0, 9.0, 13.0]
4 2 18
argmax= 2
[0.0, 0.0, 4.0]
3 1 19
[[[19, 14], [18, 14], [17, 14], [17, 13], [16, 13], [16, 12], [15, 12], [14, 11], [13, 10], [12, 9], [11, 8], [10, 7], [9, 6], [8, 6], [8, 5], [7, 5], [6, 4], [5, 3], [4, 2], [3, 1]]]


In [33]:
def extract_Alignment(indeledSequence1, indeledSequence2,alignment):
    extractedAlignment=[]
    extractedAlignment.append([(indeledSequence1[alignment[0][1]]),(indeledSequence2[alignment[0][0]])])
    prev=alignment[0]
    current=alignment[1]
    for i in range(1,len(alignment),1):
        print(alignment[i])
        if((alignment[i-1][0]==(alignment[i][0]+1)) and (alignment[i-1][1]==(alignment[i][1]+1))):
            extractedAlignment.append([(indeledSequence2[alignment[i][0]]),indeledSequence1[alignment[i][1]]])
            continue
        if(alignment[i-1][0]==(alignment[i][0]+1)):
            extractedAlignment.append([indeledSequence1[alignment[i][1]],(indeledSequence2[0])])
            continue
        
        if(alignment[i-1][1]==(alignment[i][1]+1)):
            extractedAlignment.append([indeledSequence1[0],(indeledSequence2[alignment[i][0]])])
            continue
        
    return extractedAlignment

extracted=extract_Alignment(indeledSequence1,indeledSequence2,optimale_scores[0])
print(extracted)
print(indeledSequence1)
print(indeledSequence2)

def show_extracted_Alignment(extractedAlignment):
    for residuePair in extractedAlignment:
        print(residuePair[0],end='')
    print()
    
    for residuePair in extractedAlignment:
        print('|',end='')
    print()    
    for residuePair in extractedAlignment:
        print(residuePair[1],end='')
    print()
show_extracted_Alignment(extracted)

[18, 14]
[17, 14]
[17, 13]
[16, 13]
[16, 12]
[15, 12]
[14, 11]
[13, 10]
[12, 9]
[11, 8]
[10, 7]
[9, 6]
[8, 6]
[8, 5]
[7, 5]
[6, 4]
[5, 3]
[4, 2]
[3, 1]
[['G', 'G'], ['G', '-'], ['G', '-'], ['-', 'G'], ['G', '-'], ['-', 'A'], ['A', '-'], ['C', 'C'], ['A', 'A'], ['G', 'G'], ['A', 'A'], ['T', 'T'], ['C', 'C'], ['C', '-'], ['-', 'A'], ['A', '-'], ['G', 'G'], ['C', 'C'], ['C', 'C'], ['A', 'A']]
-ACCGACTAGACAGGT
-TTACCGGACTAGACCAGCG
GGG-G-ACAGATCC-AGCCA
||||||||||||||||||||
G--G-A-CAGATC-A-GCCA


In [8]:
score_Table_Dict= fill_score_table(score_Table, score_Table_Dict)
print(score_Table_Dict)

{'-': {'-': 0, 'A': -1, 'C': -1, 'G': -1, 'T': -1}, 'A': {'-': -1, 'A': 2, 'C': -1, 'G': -1, 'T': -1}, 'C': {'-': -1, 'A': -1, 'C': 2, 'G': -1, 'T': -1}, 'G': {'-': -1, 'A': -1, 'C': -1, 'G': 2, 'T': -1}, 'T': {'-': -1, 'A': -1, 'C': -1, 'G': -1, 'T': 2}}


In [9]:
alignmentMatrix=local_alignment(sequence3, sequence4, score_Table_Dict)    
print(alignmentMatrix)

{'-': {'-': 0, 'A': -1, 'C': -1, 'G': -1, 'T': -1}, 'A': {'-': -1, 'A': 2, 'C': -1, 'G': -1, 'T': -1}, 'C': {'-': -1, 'A': -1, 'C': 2, 'G': -1, 'T': -1}, 'G': {'-': -1, 'A': -1, 'C': -1, 'G': 2, 'T': -1}, 'T': {'-': -1, 'A': -1, 'C': -1, 'G': -1, 'T': 2}}
-1.0
2.0
-1.0
0.0
C
C
{'-': {'-': 0, 'A': -1, 'C': -1, 'G': -1, 'T': -1}, 'A': {'-': -1, 'A': 2, 'C': -1, 'G': -1, 'T': -1}, 'C': {'-': -1, 'A': -1, 'C': 2, 'G': -1, 'T': -1}, 'G': {'-': -1, 'A': -1, 'C': -1, 'G': 2, 'T': -1}, 'T': {'-': -1, 'A': -1, 'C': -1, 'G': -1, 'T': 2}}
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 1. 0. 0. 2. 1.]
 [0. 0. 1. 1. 0. 2. 1. 1.]
 [0. 0. 2. 1. 0. 1. 4. 3.]
 [0. 2. 1. 4. 3. 2. 3. 3.]
 [0. 1. 1. 3. 3. 5. 4. 3.]
 [0. 0. 0. 2. 2. 4. 4. 6.]
 [0. 0. 2. 1. 1. 3. 6. 5.]]


In [10]:
indeledSequence1="-"+sequence3
indeledSequence2="-"+sequence4
max_indices=get_max_score_indices(alignmentMatrix)
print(max_indices)
optimale_scores=[]
for maxScore in max_indices:
        optimale_scores.append(optimal_path2(maxScore[0],maxScore[1],alignmentMatrix,[],0,indeledSequence1,indeledSequence2,score_Table_Dict))

print(optimale_scores)

[[6, 7], [7, 6]]
6 7 0
3.0 6.0
2.0 6.0
6.0 6.0
5 6 1
4.0 4.0
2.0 4.0
1.0 4.0
5 5 2
2.0 5.0
1.0 5.0
5.0 5.0
4 4 3
3.0 3.0
0.0 3.0
3.0 3.0
4 3 4
0.0 4.0
0.0 4.0
4.0 4.0
3 2 5
0.0 2.0
0.0 2.0
2.0 2.0
2 1 6
0.0 0.0
0.0 0.0
0.0 0.0
2 0 7
1 0 8
0 0 9
7 6 0
2.0 6.0
3.0 6.0
6.0 6.0
6 5 1
1.0 4.0
4.0 4.0
2.0 4.0
5 5 2
2.0 5.0
1.0 5.0
5.0 5.0
4 4 3
3.0 3.0
0.0 3.0
3.0 3.0
4 3 4
0.0 4.0
0.0 4.0
4.0 4.0
3 2 5
0.0 2.0
0.0 2.0
2.0 2.0
2 1 6
0.0 0.0
0.0 0.0
0.0 0.0
2 0 7
1 0 8
0 0 9
[[[6, 7], [5, 6], [5, 5], [4, 4], [4, 3], [3, 2], [2, 1], [2, 0], [1, 0], [0, 0]], [[7, 6], [6, 5], [5, 5], [4, 4], [4, 3], [3, 2], [2, 1], [2, 0], [1, 0], [0, 0]]]


In [11]:

extracted=extract_Alignment(indeledSequence1,indeledSequence2,optimale_scores[0])
print(extracted)
print(indeledSequence1)
print(indeledSequence2)

[5, 6]
[5, 5]
[4, 4]
[4, 3]
[3, 2]
[2, 1]
[2, 0]
[1, 0]
[0, 0]
[['G', 'G'], ['T', 'C'], ['T', '-'], ['A', 'A'], ['A', '-'], ['C', 'C'], ['T', 'A'], ['T', '-'], ['-', '-'], ['-', '-']]
-ACAATCG
-CTCATGC


In [12]:
show_extracted_Alignment(extracted)

GTTAACTT--
||||||||||
GC-A-CA---


In [13]:
print(matrix)

BLOSUM(62, default=-inf, {'AA': 4.0, 'AR': -1.0, 'AN': -2.0, 'AD': -2.0, 'AC': 0.0, 'AQ': -1.0, 'AE': -1.0, 'AG': 0.0, 'AH': -2.0, 'AI': -1.0, 'AL': -1.0, 'AK': -1.0, 'AM': -1.0, 'AF': -2.0, 'AP': -1.0, 'AS': 1.0, 'AT': 0.0, 'AW': -3.0, 'AY': -2.0, 'AV': 0.0, 'AB': -2.0, 'AJ': -1.0, 'AZ': -1.0, 'AX': -1.0, 'A*': -4.0, 'RA': -1.0, 'RR': 5.0, 'RN': 0.0, 'RD': -2.0, 'RC': -3.0, 'RQ': 1.0, 'RE': 0.0, 'RG': -2.0, 'RH': 0.0, 'RI': -3.0, 'RL': -2.0, 'RK': 2.0, 'RM': -1.0, 'RF': -3.0, 'RP': -2.0, 'RS': -1.0, 'RT': -1.0, 'RW': -3.0, 'RY': -2.0, 'RV': -3.0, 'RB': -1.0, 'RJ': -2.0, 'RZ': 0.0, 'RX': -1.0, 'R*': -4.0, 'NA': -2.0, 'NR': 0.0, 'NN': 6.0, 'ND': 1.0, 'NC': -3.0, 'NQ': 0.0, 'NE': 0.0, 'NG': 0.0, 'NH': 1.0, 'NI': -3.0, 'NL': -3.0, 'NK': 0.0, 'NM': -2.0, 'NF': -3.0, 'NP': -2.0, 'NS': 1.0, 'NT': 0.0, 'NW': -4.0, 'NY': -2.0, 'NV': -3.0, 'NB': 4.0, 'NJ': -3.0, 'NZ': 0.0, 'NX': -1.0, 'N*': -4.0, 'DA': -2.0, 'DR': -2.0, 'DN': 1.0, 'DD': 6.0, 'DC': -3.0, 'DQ': 0.0, 'DE': 2.0, 'DG': -1.0, 'DH': -